In [ ]:
import sys
sys.path.insert(0, r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")

from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'fact'

evaluator = get_evaluator(task)

In [ ]:
from prettytable import PrettyTable

def convert_to_json(output_list, src_list=None, ref_list=None, context_list=None, \
            scores=None, doc_id=None, system_id=None):
    """
        Convert the data into the json format.

        output_list: a list of model output
        src_list: source input for different NLG tasks. For example, source document for summarization
                  and dialogue history for dialogue response generation
        ref_list: human-annotated groundtruth
        context_list: the context needed to evaluate several specific dimension. For example,
                      additional factual information when evaluating engagingness and groundedness in dialogues
        scores: human scores for evaluating the model output. They can be used to calculate the correlation
                between evaluators and human judgements. The scores should be stored in a dictionary. For example,
                {'fluency': 2.0, 'coherence': 3.0} could be the human score for a sample.
        doc_id: the index of the input source. It can be used to calculate summary-level correlation for summarzation
        system_id: the index of the generation system. It can be used to calculate system-level correlation.
    """
    json_data = []
    for i in range(len(output_list)):
        cur = {}
        cur['system_output'] = output_list[i]
        if src_list is not None:
            cur['source'] = src_list[i]
        if ref_list is not None:
            cur['reference'] = ref_list[i]
        if context_list is not None:
            cur['context'] = context_list[i]
        if scores is not None:
            cur['scores'] = scores[i]
        if doc_id is not None:
            cur['doc_id'] = doc_id[i]
        if system_id is not None:
            cur['system_id'] = system_id[i]
        json_data.append(cur)
    return json_data


def add_question(dimension, output, src=None, ref=None, context=None, task=None):
    """
        Add questions to generate input in Bool-QA format for UniEval.

        dimension: specific dimension to be evaluated
        src: source input for different NLG tasks. For example, source document for summarization
             and dialogue history for dialogue response generation.
        output: output text generated by the models
        ref: human-annotataed groundtruth
        context: the context needed to evaluate several specific dimension. For example,
                 additional factual information when evaluating engagingness and groundedness in dialogues.
    """

    input_with_question = []
    for i in range(len(output)):
        # For summarization
        if task == 'summarization':
            if dimension == 'fluency':
                cur_input = 'question: Is this a fluent paragraph? </s> paragraph: ' + output[i]
            elif dimension == 'coherence':
                cur_input = 'question: Is this a coherent summary to the document? </s> summary: ' + output[i] + ' </s> document: ' + src[i]
            elif dimension == 'consistency':
                cur_input = 'question: Is this claim consistent with the document? </s> claim: ' + output[i] + ' </s> document: ' + src[i]
            elif dimension == 'relevance':
                cur_input = 'question: Is this summary relevant to the reference? </s> summary: ' + output[i] + ' </s> reference: ' + ref[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For dialogues
        elif task == 'dialogue':
            if dimension == 'naturalness':
                cur_input = 'question: Is this a natural response in the dialogue? </s> response: ' + output[i]
            elif dimension == 'coherence':
                cur_input = 'question: Is this a coherent response given the dialogue history? </s> response: '\
                            + output[i] + ' </s> dialogue history: ' + src[i]
            elif dimension == 'engagingness':
                cur_input = 'question: Is this an engaging and informative response according to the dialogue history and fact? </s> response: '\
                            + output[i] + ' </s> dialogue history: ' + src[i] + ' </s> fact: ' + context[i]
            elif dimension == 'groundedness':
                cur_input = 'question: Is this response consistent with knowledge in the fact? </s> response: '\
                            + output[i] + ' </s> fact: ' + context[i]
            elif dimension == 'understandability':
                cur_input = 'question: Is this an understandable response in the dialogue? </s> response: ' + output[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For data-to-text
        elif task == 'data2text':
            if dimension == 'naturalness':
                cur_input = 'question: Is this a fluent utterance? </s> utterance: ' + output[i]
            elif dimension == 'informativeness':
                cur_input = 'question: Is this sentence informative according to the reference? </s> sentence: '\
                            + output[i] + ' </s> reference: ' + ref[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For factual consistency detection
        elif task == 'fact':
            if dimension == 'consistency':
                cur_input = 'question: Is this claim consistent with the document? </s> claim: ' + output[i] + ' </s> document: ' + src[i]
            else:
                raise NotImplementedError('No other dimensions for the factual consistency detection task.')
        # For new customized tasks
        else:
            raise NotImplementedError('Other tasks are not implemented, please customize specific tasks here.')
        input_with_question.append(cur_input)
    return input_with_question


def print_scores(scores):
    table = PrettyTable(['Dimensions','Score'])
    print('\nEvaluation scores are shown below:')
    dims = list(scores[0].keys())
    for dim in dims:
        cur_score = 0
        for i in range(len(scores)):
            cur_score += scores[i][dim]
        table.add_row([dim, round(cur_score / len(scores), 6)])
    print(table)

In [ ]:
import numpy as np
from nltk import sent_tokenize
from scorer import UniEvaluator  # Make sure this import works after placing scorer.py in the same directory

def evaluate(data, dims=None, overall=True, print_result=False, model_name_or_path="t5-small", task='summarization', device='cuda:0', individual=True):
    """
    Get the scores of all the given dimensions (fluency, consistency, coherence, relevance)

    data: A list of dictionaries, where each dictionary contains:
          - 'source': The original text
          - 'system_output': The generated system output (summary)
          - 'reference' (optional): Reference summary for relevance evaluation

    dims: A list of dimensions to be evaluated. If dims is None, it evaluates four default dimensions:
          coherence, consistency, fluency, relevance.

    overall: Boolean to indicate whether the overall score is calculated as the average of all dimensions.

    print_result: Boolean to print the results on the screen.

    model_name_or_path: The model name or path to use for evaluation, e.g., 't5-small'

    task: The task type (used in scoring if needed, like summarization or other NLP tasks).

    device: The device to use for evaluation ('cpu' or 'cuda:0').
    """

    # Instantiate the scorer
    scorer = UniEvaluator(model_name_or_path=model_name_or_path, device=device)

    n_data = len(data)
    eval_scores = [{} for _ in range(n_data)]

    # Default dimensions if not provided
    if dims is None:
        dims = ['coherence', 'consistency', 'fluency', 'factual consistency']   #add relevance

    for dim in dims:
        print(f'Evaluating {dim} of {n_data} samples !!!')

        if dim == 'consistency' or dim == 'fluency':
            # Sentence-level scores for consistency and fluency
            src_list, output_list = [], []
            n_sents = []  # number of sentences in each summary

            for i in range(n_data):
                if dim == 'consistency':
                    source = data[i]['source']
                else:
                    source = ''
                system_outputs = sent_tokenize(data[i]['system_output'])
                n_sents.append(len(system_outputs))
                for j in range(len(system_outputs)):
                    src_list.append(source)
                    output_list.append(system_outputs[j])

            input_list = add_question(dimension=dim, output=output_list, src=src_list, task=task)
            sent_score = scorer.score(input_list)

            # Calculate average sentence-level scores for each sample
            start_idx = 0
            score = []
            for cur_n_sent in n_sents:
                score.append(sum(sent_score[start_idx:start_idx + cur_n_sent]) / cur_n_sent)
                start_idx += cur_n_sent

        elif dim == 'coherence' or dim == 'relevance':
            # Summary-level scores for coherence and relevance
            src_list, output_list, ref_list = [], [], []

            for i in range(n_data):
                src_list.append(data[i]['source'])
                output_list.append(data[i]['system_output'])
                if dim == 'relevance':
                    ref_list.append(data[i]['reference'])

            input_list = add_question(dimension=dim, output=output_list, src=src_list, ref=ref_list, task=task)
            score = scorer.score(input_list)

        elif dim == 'factual consistency':
            output_list, src_list = [], []

            for i in range(n_data):
                src_list.append(data[i]['source'])
                output_list.append(data[i]['system_output'])

            data = convert_to_json(output_list=output_list, src_list=src_list)
            eval_score = evaluator.evaluate(data)
            score = []

            for i in eval_score:
                temp = i['consistency']
                score.append(temp)

        else:
            raise NotImplementedError(f"The input format for the dimension '{dim}' is still undefined. Please customize it.")

        # Store the scores for the current dimension
        for i in range(n_data):
            eval_scores[i][dim] = score[i]

    # Calculate overall score (average of all evaluated dimensions)
    if overall:
        for i in range(n_data):
            eval_scores[i]['overall'] = np.mean([eval_scores[i][dim] for dim in dims])

    # Print the result if requested
    if print_result:
        print_scores(eval_scores)

    if individual:
        individual_scores = []
        for i in range(n_data):
            temp = [eval_scores[i][dim] for dim in dims]
            individual_scores.append(temp)

        return np.array(individual_scores)

    # Calculate average score across all the dimensions except 'overall'
    avg_score = []
    for i in range(n_data):
        # Exclude 'overall' from the averaging
        dimensions = [dim for dim in dims if dim != 'overall']
        avg_score.append(np.mean([eval_scores[i][dim] for dim in dimensions]))

    return avg_score

In [ ]:
# # Import all required libraries
# import torch
# import transformers
# from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments
# from trl import RewardTrainer, PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
# from datasets import Dataset
# import json
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, TaskType
# import bitsandbytes as bnb
# import numpy as np
# from tqdm import tqdm
# import time
# import sys

# # Add UniEval to path and import
# sys.path.append(r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")
# from utils import convert_to_json
# from metric.evaluator import get_evaluator

# # Configuration
# DATA_PATH = "combined_clinical_notes.csv"
# # MODEL_PATH = r"D:\kshitij-weights-folder\qwen-aloe-9-4-base-fine-tune"
# MODEL_PATH = "gpt2" 
# # PEFT_ADAPTER_PATH = r"D:\kshitij-weights-folder\qwen-aloe-9-4-base-fine-tune-peft-adapaters"
# MEDICAL_PROMPT = "\nGenerate a concise medical summary focusing on key findings and treatment plans:"

# # Load and prepare data
# df = pd.read_csv(DATA_PATH)
# train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
# eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
# dataset = Dataset.from_pandas(eval_df.rename(columns={"dialogue": "review"}))

# # Tokenizer setup
# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
# tokenizer.pad_token = tokenizer.eos_token

# # Dataset preprocessing
# def preprocess_function(examples):
#     return {
#         "input_ids": tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512),
#         "query": tokenizer.decode(tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512), skip_special_tokens=True)
#     }

# dataset = dataset.map(preprocess_function, batched=False)
# dataset.set_format("pytorch")

# # Model configuration
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     MODEL_PATH,
#     quantization_config=bnb_config,
#     device_map="auto"
# )
# base_model = prepare_model_for_kbit_training(base_model)

# # PEFT/LoRA configuration
# lora_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,
#     r=8,
#     lora_alpha=32,
#     lora_dropout=0.1,
#     bias="none",
#     target_modules=["c_attn", "c_proj"],
# )

# # model_with_lora = get_peft_model(base_model, lora_config)
# model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model, peft_config=lora_config).to("cuda")

# # Reference model
# ref_model = create_reference_model(model).to("cuda")
# ref_model.eval()
# for param in ref_model.parameters():
#     param.requires_grad = False

# NUM_CANDIDATES = 2

# # PPO Configuration
# ppo_config = PPOConfig(
#     model_name=MODEL_PATH,
#     ppo_epochs=1,
#     gradient_accumulation_steps=1,
#     steps=5,
#     batch_size=1*NUM_CANDIDATES,
#     mini_batch_size=1*NUM_CANDIDATES,
#     learning_rate=2e-5,
#     log_with=None,
#     # project_kwargs={"logging_dir": r"D:\kshitij-weights-folder\gpt2-rl-logs"}
# )

# # optimizer = torch.optim.AdamW(
# #     filter(lambda p: p.requires_grad, ppo_model.parameters()),
# #     lr=2e-5,
# #     eps=1e-5,  # Slightly larger epsilon for stability
# # )

# # Initialize PPO Trainer
# ppo_trainer = PPOTrainer(
#     config=ppo_config,
#     model=model,
#     ref_model=ref_model,
#     tokenizer=tokenizer,
#     dataset=dataset,
#     optimizer=bnb.optim.Adam8bit(model.parameters(), lr=ppo_config.learning_rate)
# )

# # Evaluation setup
# # sum_eval = get_evaluator("summarization", "cuda"="cuda" if torch.cuda.is_available() else "cpu")

# def get_score(src, res):
#     # weights = np.array([0.1, 0.2, 0.3, 0.4])  # coherence, consistency, fluency, factual consistency
#     # sample_data = [{"source": q, "system_output": r} for q, r in zip(game_data["query"], game_data["response"])]
    
#     # scores = sum_eval.evaluate(sample_data, overall=False)
#     # scores = evaluate(sample_data, overall=False)
#     # weighted_scores = []
    
#     # for dimension_scores in scores:
#     #     adjusted = np.where(
#     #         dimension_scores < 0.5,
#     #         -dimension_scores * weights,
#     #         dimension_scores * weights
#     #     )
#     #     weighted_scores.append(torch.tensor(np.sum(adjusted)/4, dtype=torch.float32).to(model.pretrained_model.device))
    
#     # return weighted_scores

#     data = convert_to_json(
#         output_list=res,
#         src_list=src,
#     )
#     # raw = sum_eval.evaluate(data, print_result=True)
#     # dims = ['coherence', 'consistency', 'fluency', 'factual consistency']
#     raw = evaluate(data, overall=False)
#     score = [
#         [d[0], d[1], d[2], d[3]]
#         for d in raw
#     ]
#     scores = torch.tensor(score, dtype=torch.float32).numpy()  # CPU (B,4

#     k = len(res)
#     dom_counts = np.zeros(k)
    
#     for i in range(k):
#         for j in range(k):
#             if i == j:
#                 continue
#             # Check dominance: i dominates j if all scores are >= and at least one is >
#             if np.all(scores[i] >= scores[j]) and np.any(scores[i] > scores[j]):
#                 dom_counts[i] += 1
    
#     # Convert to [-1, 1] range reward
#     max_dom = k - 1
#     if max_dom > 0:
#         rewards = 2 * (dom_counts / max_dom) - 1
#     else:
#         rewards = np.zeros(k)
    
#     return rewards


# max_position_embeddings = model.pretrained_model.config.max_position_embeddings 
# # Training loop
# generation_kwargs = {
#     "min_length": -1,
#     "top_k": 0.0,
#     "top_p": 1.0,
#     "do_sample": True,
#     "pad_token_id": tokenizer.eos_token_id,
#     # "max_new_tokens": 64,  # Increased for better summary generation
#     "eos_token_id": -1,
#     "max_length": max_position_embeddings,
#     "max_new_tokens": 64
# }
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# # Use consistent compute dtype
# COMPUTE_DTYPE = torch.float32  # Using float32 to avoid dtype issues

# for epoch in range(1):
#     for batch_idx, batch in enumerate(tqdm(ppo_trainer.dataloader)):
#         (logs, game_data,) = (
#             dict(),
#             dict(),
#         )

#         # task_list = choices(ctrl_str, k=config.batch_size)
#         # game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
#         game_data["query"] = [q for q in batch["query"]]
#         # query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]
#         query_tensors = [input_ids for input_ids in batch["input_ids"]]
#         all_outs = []
        
#         response_tensors = []
#         for query in query_tensors:
#             for _ in range(NUM_CANDIDATES):
#                 original_notes = tokenizer.decode(query)
                
#                 # Combine with medical prompt only during generation
#                 full_prompt = f"{MEDICAL_PROMPT}{original_notes}" #TRy with full prompt here
#                 full_prompt_tensor = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda").squeeze(0)
                
#                 with torch.no_grad():
#                     response = ppo_trainer.generate(
#                         full_prompt_tensor,
#                         **generation_kwargs
#                     )
#                 # Ensure response doesn't exceed max length
#                 response = response[:, :generation_kwargs["max_new_tokens"]]
#                 all_outs.append(response)
#                 response_tensors.append(response.squeeze())

#             outs = torch.stack(all_outs, dim=1)
#             B, K, _ = outs.shape
            
#             # Decode outputs for evaluation
#             hyps = []
#             for b in range(B):
#                 hyps_b = []
#                 for k in range(K):
#                     try:
#                         text = tokenizer.decode(outs[b, k], skip_special_tokens=True)
#                         hyps_b.append(text)
#                     except Exception as e:
#                         print(f"Error decoding text: {e}")
#                         hyps_b.append("")  # Add empty string as fallback
#                 hyps.append(hyps_b)

#             # rewards = []
#             # for b in range(len(batch['input_ids'])):
#             #     # Get scores for all candidates (K, 4)
#             #     scores = get_score(
#             #         batch['query'][b] * NUM_CANDIDATES,
#             #         hyps[b]
#             #     ).numpy()

#             #     print("check")

#             #     dom_counts = np.zeros(NUM_CANDIDATES)
#             #     for i in range(NUM_CANDIDATES):
#             #         for j in range(NUM_CANDIDATES):
#             #             if i == j:
#             #                 continue
#             #             # Check if i dominates j
#             #             if np.all(scores[i] >= scores[j]) and np.any(scores[i] > scores[j]):
#             #                 dom_counts[i] += 1

#             #     max_dom = NUM_CANDIDATES - 1
#             #     scalar_rewards = 2 * (dom_counts / max_dom) - 1
#             #     rewards.append(scalar_rewards)

#             # flat_queries = []
#             # flat_responses = []
#             # flat_rewards = []

#             # for b in range(len(batch['input_ids'])):
#             #     for k in range(NUM_CANDIDATES):
#             #         flat_queries.append(batch['input_ids'][b])
#             #         flat_responses.append(outs[b, k])
#             #         flat_rewards.append(torch.tensor([rewards[b][k]], device="cuda"))

#             # stats = ppo_trainer.step(
#             #     queries   = flat_queries,    # e.g. [ q0, q0 ]
#             #     responses = flat_responses,  # e.g. [ r0, r1 ]
#             #     scores    = flat_rewards     # e.g. [ s0, s1 ]
#             # )

#             flat_queries, flat_responses, flat_rewards = [], [], []
                    
#             for b in range(B):
#                 try:
#                     # Calculate rewards
#                     rewards_b = get_score(
#                         [batch['query'][b]] * K, 
#                         hyps[b],
#                     )
                    
#                     # Flatten for PPO
#                     for k in range(K):
#                         flat_queries.append(batch['input_ids'][b])
#                         flat_responses.append(outs[b, k])
#                         flat_rewards.append(torch.tensor([rewards_b[k]], device=DEVICE, dtype=COMPUTE_DTYPE))
                        
#                 except Exception as e:
#                     print(f"Error computing rewards: {e}")
#                     continue
            
#             # Safety check
#             if len(flat_queries) != ppo_config.batch_size:
#                 print(f"Batch size mismatch: expected {ppo_config.batch_size}, got {len(flat_queries)}")
#                 continue

#             try:
#                 # Verify shapes match
#                 print(f"Queries: {len(flat_queries)}, Responses: {len(flat_responses)}, Rewards: {len(flat_rewards)}")
                
#                 # Manual memory management
#                 # torch.cuda.empty_cache()
                
#                 # Do PPO step
#                 stats = ppo_trainer.step(
#                     queries=flat_queries,
#                     responses=flat_responses,
#                     scores=flat_rewards
#                 )
                
#                 # Success! Log the output
#                 print(f"Batch {batch_idx} PPO step successful!")
#                 print(f"Sample output: {hyps[0][0][:100]}...")
#                 avg_reward = np.mean([r.item() for r in flat_rewards])
#                 print(f"Average reward: {avg_reward:.4f}")
                
#             except RuntimeError as e:
#                 print(f"Error in PPO step: {e}")
                
#                 # If still running into CUDA errors, try moving to CPU
#                 if "CUDA" in str(e) and DEVICE != "cpu":
#                     print("\nContinuing to encounter CUDA errors. Try two options:")
#                     print("1. Change DEVICE = 'cpu' at the top of the script")
#                     print("2. Or use the non-quantized model version\n")
                    
#                 # Clear memory and continue
#                 # if torch.cuda.is_available():
#                 #     torch.cuda.empty_cache()
            
#             # if batch_idx % 10 == 0:
#             #     print(f"Epoch {epoch+1}, Batch {batch_idx}")
#             #     print(f"Sample output: {hyps[0][0][:100]}...")
#             #     print(f"Average reward: {np.mean([r.item() for r in flat_rewards]):.4f}")

#     print(f"✅ Epoch {epoch+1}/3 complete")
    
# print("🎉 PPO fine-tuning done")
# #         print(response_tensors)
#         # game_data["response"] = [tokenizer.decode(r) for r in response_tensors]

#         # print("check")

#         # texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
#         # logits = get_score(game_data)
#         # rewards = logits
#         # rewards = pos_logit_to_reward(logits, task_list)
#         # rewards = [torch.tensor([1.0], device=query_tensors[0].device) for _ in range(len(texts))]

#         #### Run PPO trainings
#         # t = time.time()
#         # stats = ppo_trainer.step(query_tensors, response_tensors, rewards)


In [1]:
# Import all required libraries
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments
from trl import RewardTrainer, PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from datasets import Dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, TaskType
import bitsandbytes as bnb
import numpy as np
import os
from tqdm import tqdm
import time
import sys
# Add UniEval to path and import
sys.path.append(r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")
from utils import convert_to_json
from metric.evaluator import get_evaluator

# Configuration
DATA_PATH = "combined_clinical_notes.csv"
MODEL_PATH = "gpt2" 
EXTRACTION_MODEL_PATH = "bigscience/bloomz-1b7"  # Model for extracting key information
# Enhanced prompt with more guidance
MEDICAL_PROMPT = "\nGenerate a concise medical summary focusing on key findings and treatment plans. Include chief complaints, symptoms, medications, and recommendations if mentioned in the conversation:"

# Initialize the extraction model
print(f"Loading extraction model from {EXTRACTION_MODEL_PATH}...")
extraction_tokenizer = AutoTokenizer.from_pretrained(EXTRACTION_MODEL_PATH)
extraction_model = AutoModelForCausalLM.from_pretrained(
    EXTRACTION_MODEL_PATH,
    torch_dtype=torch.float16,  # Use fp16 for efficiency
    device_map="auto"
)

def extract_medical_info(conversation, max_length=300):
    """Use the extraction model to pull out key medical information"""
    extraction_prompt = (
        "Extract these key medical information points from the conversation below:\n"
        "- Chief complaint\n"
        "- Symptoms and duration\n"
        "- Current medications\n"
        "- Vital signs\n"
        "- Physical exam findings\n"
        "- Recommendations\n\n"
        f"Conversation:\n{conversation}\n\n"
        "Extracted information:"
    )
    
    # Truncate long conversations if needed
    if len(extraction_prompt.split()) > 800:
        conversation_words = conversation.split()
        truncated_conversation = " ".join(conversation_words[:700])
        extraction_prompt = extraction_prompt.replace(conversation, truncated_conversation)
    
    inputs = extraction_tokenizer(extraction_prompt, return_tensors="pt").to(extraction_model.device)
    
    with torch.no_grad():
        try:
            outputs = extraction_model.generate(
                inputs.input_ids,
                max_length=len(inputs.input_ids[0]) + max_length,  # Allow for generated content
                temperature=0.3,  # Lower temperature for more deterministic extraction
                top_p=0.95
            )
            
            extracted_info = extraction_tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Remove the prompt part
            if extraction_prompt in extracted_info:
                extracted_info = extracted_info.replace(extraction_prompt, "").strip()
            else:
                # Try to find a reasonable place to split the text
                # This helps when the model modifies the prompt slightly
                prompt_end = "Extracted information:"
                if prompt_end in extracted_info:
                    extracted_info = extracted_info.split(prompt_end)[1].strip()
            
            return extracted_info
            
        except Exception as e:
            print(f"Error in extraction model: {e}")
            # Return a simplified fallback extraction
            return "Unable to extract detailed information. Please see the original conversation."

# Load and prepare data
df = pd.read_csv(DATA_PATH)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
dataset = Dataset.from_pandas(eval_df.rename(columns={"dialogue": "review"}))

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

# Dataset preprocessing
def preprocess_function(examples):
    return {
        "input_ids": tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512),
        "query": tokenizer.decode(tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512), skip_special_tokens=True)
    }

dataset = dataset.map(preprocess_function, batched=False)
dataset.set_format("pytorch")

# Model configuration
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model = prepare_model_for_kbit_training(base_model)

# PEFT/LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"],
)

model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model, peft_config=lora_config).to("cuda")

# Reference model
ref_model = create_reference_model(model).to("cuda")
ref_model.eval()
for param in ref_model.parameters():
    param.requires_grad = False

NUM_CANDIDATES = 2

# PPO Configuration
ppo_config = PPOConfig(
    model_name=MODEL_PATH,
    ppo_epochs=1,
    gradient_accumulation_steps=1,
    steps=5,
    batch_size=1*NUM_CANDIDATES,
    mini_batch_size=1*NUM_CANDIDATES,
    learning_rate=2e-5,
    log_with=None,
)

# Initialize PPO Trainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    optimizer=bnb.optim.Adam8bit(model.parameters(), lr=ppo_config.learning_rate)
)

# Initialize evaluation model
sum_eval = get_evaluator("summarization", device="cpu")

# Evaluation function
def evaluate(data, overall=False):
    """Wrapper around sum_eval.evaluate to handle errors"""
    try:
        return sum_eval.evaluate(data, overall=overall)
    except Exception as e:
        print(f"Error in evaluate: {e}")
        # Return default scores if evaluation fails
        return [[0.5, 0.5, 0.5, 0.5]] * len(data)

def get_score(src, res):
    """Calculate rewards based on evaluation scores"""
    data = convert_to_json(
        output_list=res,
        src_list=src,
    )
    
    raw = evaluate(data, overall=False)
    score = [
        [d[0], d[1], d[2], d[3]]
        for d in raw
    ]
    scores = np.array(score, dtype=np.float32)
    k = len(res)
    dom_counts = np.zeros(k)
    
    for i in range(k):
        for j in range(k):
            if i == j:
                continue
            # Check dominance: i dominates j if all scores are >= and at least one is >
            if np.all(scores[i] >= scores[j]) and np.any(scores[i] > scores[j]):
                dom_counts[i] += 1
    
    # Convert to [-1, 1] range reward
    max_dom = k - 1
    if max_dom > 0:
        rewards = 2 * (dom_counts / max_dom) - 1
    else:
        rewards = np.zeros(k)
    
    return rewards

max_position_embeddings = model.pretrained_model.config.max_position_embeddings 

# Training loop with improved generation parameters
generation_kwargs = {
    "min_length": -1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": -1,
    "max_length": max_position_embeddings,
    "max_new_tokens": 96,
    "temperature": 0.7
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
COMPUTE_DTYPE = torch.float32  # Using float32 to avoid dtype issues

# Track extract-summarize cache to avoid recomputing
extract_cache = {}

for epoch in range(3):
    for batch_idx, batch in enumerate(tqdm(ppo_trainer.dataloader)):
        logs, game_data = dict(), dict()
        game_data["query"] = [q for q in batch["query"]]
        query_tensors = [input_ids for input_ids in batch["input_ids"]]
        
        # Process one query at a time to prevent batch size mismatch
        for query_idx, query in enumerate(query_tensors):
            try:
                # Skip empty or very short queries
                if len(query) < 10:
                    print(f"Skipping query {query_idx}: too short")
                    continue
                
                # Generate NUM_CANDIDATES responses for this query
                responses = []
                decoded_responses = []
                
                # Get original text and extract key information
                original_notes = tokenizer.decode(query)
                
                # Check cache first to avoid redundant extractions
                if original_notes in extract_cache:
                    extracted_info = extract_cache[original_notes]
                else:
                    extracted_info = extract_medical_info(original_notes)
                    extract_cache[original_notes] = extracted_info
                
                # Construct enhanced prompt with extracted information
                full_prompt = (
                    f"{MEDICAL_PROMPT}\n"
                    f"KEY INFORMATION:\n{extracted_info}\n\n"
                    "Based on the above, generate a concise medical summary:"
                )
                
                # Encode the full prompt
                full_prompt_tensor = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda").squeeze(0)
                
                for _ in range(NUM_CANDIDATES):
                    with torch.no_grad():
                        response = ppo_trainer.generate(
                            full_prompt_tensor,
                            **generation_kwargs
                        )
                    # Ensure response doesn't exceed max length
                    response = response[:, :generation_kwargs["max_new_tokens"]]
                    responses.append(response.squeeze())
                    
                    # Decode for evaluation
                    text = tokenizer.decode(response.squeeze(), skip_special_tokens=True)
                    
                    # Extract just the generated part (often it repeats the prompt)
                    if full_prompt in text:
                        generated_text = text.split(full_prompt)[1].strip()
                    else:
                        generated_text = text
                        
                    decoded_responses.append(generated_text)
                
                # Calculate rewards
                rewards = get_score(
                    [game_data["query"][query_idx]] * NUM_CANDIDATES, 
                    decoded_responses
                )
                
                # Prepare data for PPO step
                flat_queries = [query] * NUM_CANDIDATES
                flat_responses = responses
                flat_rewards = [torch.tensor([r], device=DEVICE, dtype=COMPUTE_DTYPE) for r in rewards]
                
                # Verify sizes match
                if len(flat_queries) == NUM_CANDIDATES and len(flat_responses) == NUM_CANDIDATES and len(flat_rewards) == NUM_CANDIDATES:
                    # Do PPO step for this query
                    stats = ppo_trainer.step(
                        queries=flat_queries,
                        responses=flat_responses,
                        scores=flat_rewards
                    )
                    
                    print(f"Query {query_idx} in batch {batch_idx} - PPO step successful!")
                    print(f"Extracted Info: {extracted_info[:100]}...")
                    print(f"Sample output: {decoded_responses[0][:100]}...")
                    avg_reward = np.mean([r.item() for r in flat_rewards])
                    print(f"Average reward: {avg_reward:.4f}")
                else:
                    print(f"Skipping query {query_idx} due to size mismatch: queries={len(flat_queries)}, responses={len(flat_responses)}, rewards={len(flat_rewards)}")
                    
            except Exception as e:
                print(f"Error processing query {query_idx}: {e}")
                continue
        
        # Save checkpoint after each batch
        if batch_idx % 5 == 0:
            checkpoint_path = f"D:/kshitij-weights-folder/gpt-2-tuned-ppo-checkpoints/batch_{batch_idx}"
            try:
                os.makedirs(checkpoint_path, exist_ok=True)
                ppo_trainer.model.pretrained_model.save_pretrained(checkpoint_path)
                tokenizer.save_pretrained(checkpoint_path)
                print(f"Checkpoint saved to {checkpoint_path}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")
                
    print(f"✅ Epoch {epoch+1}/3 complete")
    
    # Save epoch checkpoint
    epoch_path = f"D:/kshitij-weights-folder/gpt-2-tuned-ppo-epochs/epoch_{epoch+1}"
    os.makedirs(epoch_path, exist_ok=True)
    ppo_trainer.model.pretrained_model.save_pretrained(epoch_path)
    tokenizer.save_pretrained(epoch_path)
    print(f"Epoch checkpoint saved to {epoch_path}")
    
print("🎉 PPO fine-tuning done")

# Save the final model
save_path = r"D:\kshitij-weights-folder\gpt-2-tuned-ppo-extracted"
ppo_trainer.model.pretrained_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Loading extraction model from bigscience/bloomz-1b7...


C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0%|                                                                                 | 0/46 [00:00<?, ?it/s]C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generatio

Evaluating coherence of 2 samples !!!



100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


...

C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\trl\trainer\ppo_trainer.py:1266: UserWarning: KL divergence is starting to become negative: -309.50 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


Query 1 in batch 45 - PPO step successful!
Extracted Info: Symptoms and duration...
Sample output: if:
expiratory thoughts:
yn:[relevant relenged no symptoms orexing.P inpiration:enging discussion[en...
Average reward: -1.0000


100%|████████████████████████████████████████████████████████████████████████| 46/46 [16:00<00:00, 20.87s/it]

Checkpoint saved to D:/kshitij-weights-folder/gpt-2-tuned-ppo-checkpoints/batch_45
✅ Epoch 3/3 complete


Epoch checkpoint saved to D:/kshitij-weights-folder/gpt-2-tuned-ppo-epochs/epoch_3
🎉 PPO fine-tuning done
Model saved to D:\kshitij-weights-folder\gpt-2-tuned-ppo-extracted


In [ ]:
ppo_trainer.model.pretrained_model.save_pretrained("D:\kshitij-weights-folder\gpt-2-tuned-ppo")
tokenizer.save_pretrained("D:\kshitij-weights-folder\gpt-2-tuned-ppo")


In [5]:
# ════════════════════════════════════════════════════════════════
# Enhanced Testing Script for Reward-Based Trained Model
# ════════════════════════════════════════════════════════════════
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import time

# Create results directory
timestamp = time.strftime("%Y%m%d-%H%M%S")
RESULTS_DIR = f"evaluation_results_{timestamp}"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Check if CUDA is available for inference
CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = "cuda" if CUDA_AVAILABLE else "cpu"
print(f"Using device: {DEVICE} for inference")

# ────────────────────────────────────────────────────────────────
# 1) Load the trained model weights
# ────────────────────────────────────────────────────────────────
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Path to the saved model weights - try all promising models
MODEL_PATHS = [
    r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\reward_model_best_20250504-131132\best_model",  # Best model if available
]

# Try to load the best available model
loaded_model_path = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        loaded_model_path = path
        break

if not loaded_model_path:
    raise ValueError("No valid model path found. Please check the model paths.")

print(f"Loading model from {loaded_model_path}")
tokenizer = GPT2Tokenizer.from_pretrained(loaded_model_path)
model = GPT2LMHeadModel.from_pretrained(loaded_model_path)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"  # Important for decoder-only models
model = model.to(DEVICE)
model.eval()  # Set to evaluation mode

# Define prompt templates to test - using the same ones from training
PROMPT_TEMPLATES = [
    "Summarize the following conversation:\n\n",
    "Generate a concise medical summary of the conversation:\n\n",
    "Extract key medical information from the following conversation:\n\n",
    "Provide a summary of this medical dialogue:\n\n"
]

# ────────────────────────────────────────────────────────────────
# 2) Prepare test dataset with more samples
# ────────────────────────────────────────────────────────────────
# Load test data
df = pd.read_csv(r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\combined_clinical_notes.csv")
test_size = 30  # Using more samples for more robust evaluation
test_df = df.sample(test_size, random_state=42)  # Fixed random state for reproducibility

# Prepare test dialogues and references
dialogues = test_df["dialogue"].tolist()
references = test_df["note"].tolist()

# ────────────────────────────────────────────────────────────────
# 3) Generate summaries with multiple generation strategies
# ────────────────────────────────────────────────────────────────
print("Generating summaries with multiple strategies...")

# We'll test different generation strategies
generation_strategies = {
    "greedy": {
        "description": "Greedy Decoding",
        "params": {
            "do_sample": False,
            "num_beams": 1,
            "max_new_tokens": 128,
            "no_repeat_ngram_size": 3,
        }
    },
    "beam": {
        "description": "Beam Search",
        "params": {
            "do_sample": False,
            "num_beams": 5,
            "max_new_tokens": 128,
            "no_repeat_ngram_size": 3,
        }
    },
    "sample": {
        "description": "Sampling",
        "params": {
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.92,
            "top_k": 50,
            "max_new_tokens": 128,
            "no_repeat_ngram_size": 3,
        }
    }
}

# Try all prompt templates with best generation strategy
prompt_results = {}
for template_name, template in zip(["basic", "medical", "extract", "summary"], PROMPT_TEMPLATES):
    print(f"\nGenerating with template: {template_name}")
    
    batch_size = 4
    num_samples = len(dialogues)
    num_batches = (num_samples + batch_size - 1) // batch_size
    predictions = []
    
    for i in tqdm(range(num_batches)):
        start, end = i*batch_size, min((i+1)*batch_size, num_samples)
        convs = dialogues[start:end]
        
        # Create prompts with this template
        prompts = [
            f"{template}{c}"
            for c in convs if len(str(c).strip()) > 10
        ]
        
        if not prompts:
            continue
        
        # Tokenize
        enc = tokenizer(
            prompts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(DEVICE)
        
        # Use beam search for best quality
        with torch.no_grad():
            out_ids = model.generate(
                input_ids=enc.input_ids,
                attention_mask=enc.attention_mask,
                **generation_strategies["beam"]["params"]
            )
        
        # Decode
        dec = tokenizer.batch_decode(out_ids, skip_special_tokens=True)
        
        # Extract only the generated part (not including the prompt)
        cleaned_predictions = []
        for p, original_prompt in zip(dec, prompts):
            # Try to extract just the generated part
            if original_prompt in p:
                generated_part = p[p.find(original_prompt) + len(original_prompt):]
                cleaned_predictions.append(generated_part.strip())
            else:
                # If we can't find the prompt, use the whole generation
                cleaned_predictions.append(p.strip())
        
        predictions.extend(cleaned_predictions)
    
    prompt_results[template_name] = predictions[:num_samples]  # Ensure same length

# ────────────────────────────────────────────────────────────────
# 4) Evaluate with UniEval
# ────────────────────────────────────────────────────────────────
import sys
sys.path.append(r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")
from utils import convert_to_json
from metric.evaluator import get_evaluator

# Keep UniEval on CPU (more stable)
sum_eval = get_evaluator("summarization", device="cuda")

# Evaluate each prompt template
template_scores = {}

for template_name, predictions in prompt_results.items():
    print(f"\nEvaluating template: {template_name}")
    
    # Trim predictions and references to the same length
    min_len = min(len(predictions), len(references), len(dialogues))
    template_predictions = predictions[:min_len]
    template_references = references[:min_len]
    template_dialogues = dialogues[:min_len]
    
    # Create JSON data for UniEval
    data = convert_to_json(
        src_list=[str(d) for d in template_dialogues],
        ref_list=[str(r) for r in template_references],
        output_list=[str(p) for p in template_predictions]
    )
    
    # Run evaluation
    print(f"Running UniEval for {template_name}...")
    scores = sum_eval.evaluate(data, print_result=True)
    template_scores[template_name] = scores

# ────────────────────────────────────────────────────────────────
# 5) Find the best template and analyze results
# ────────────────────────────────────────────────────────────────
template_avgs = {}

for template_name, scores in template_scores.items():
    coherence_scores = [item["coherence"] for item in scores]
    consistency_scores = [item["consistency"] for item in scores]
    fluency_scores = [item["fluency"] for item in scores]
    relevance_scores = [item["relevance"] for item in scores]
    
    # Calculate averages
    avg_coherence = sum(coherence_scores) / len(coherence_scores)
    avg_consistency = sum(consistency_scores) / len(consistency_scores)
    avg_fluency = sum(fluency_scores) / len(fluency_scores)
    avg_relevance = sum(relevance_scores) / len(relevance_scores)
    overall_avg = (avg_coherence + avg_consistency + avg_fluency + avg_relevance) / 4
    
    template_avgs[template_name] = {
        "coherence": avg_coherence,
        "consistency": avg_consistency,
        "fluency": avg_fluency,
        "relevance": avg_relevance,
        "overall": overall_avg
    }
    
    print(f"\n=== {template_name.title()} Template Results ===")
    print(f"Average Coherence: {avg_coherence:.4f}")
    print(f"Average Consistency: {avg_consistency:.4f}")
    print(f"Average Fluency: {avg_fluency:.4f}")
    print(f"Average Relevance: {avg_relevance:.4f}")
    print(f"Overall Average: {overall_avg:.4f}")

# Find the best template
best_template = max(template_avgs.items(), key=lambda x: x[1]["overall"])
print(f"\n🏆 Best Template: {best_template[0].title()} with Overall Score: {best_template[1]['overall']:.4f}")

# Use the best template's predictions and scores for the rest of the analysis
best_predictions = prompt_results[best_template[0]]
best_scores = template_scores[best_template[0]]

# ────────────────────────────────────────────────────────────────
# 6) Visualize results with enhanced charts
# ────────────────────────────────────────────────────────────────
# Set up the plotting style
plt.style.use('seaborn-v0_8-whitegrid')

# Prepare data for visualization
template_names = list(template_avgs.keys())
coherence_avgs = [data["coherence"] for data in template_avgs.values()]
consistency_avgs = [data["consistency"] for data in template_avgs.values()]
fluency_avgs = [data["fluency"] for data in template_avgs.values()]
relevance_avgs = [data["relevance"] for data in template_avgs.values()]
overall_avgs = [data["overall"] for data in template_avgs.values()]

# Create comparison bar chart
metrics = ['Coherence', 'Consistency', 'Fluency', 'Relevance', 'Overall']
x = np.arange(len(template_names))
width = 0.15

fig, ax = plt.figure(figsize=(14, 8)), plt.subplot(111)
rects1 = ax.bar(x - 2*width, coherence_avgs, width, label='Coherence', color='#3274A1')
rects2 = ax.bar(x - width, consistency_avgs, width, label='Consistency', color='#E1812C')
rects3 = ax.bar(x, fluency_avgs, width, label='Fluency', color='#3A923A')
rects4 = ax.bar(x + width, relevance_avgs, width, label='Relevance', color='#C03D3E')
rects5 = ax.bar(x + 2*width, overall_avgs, width, label='Overall', color='#9372B2')

ax.set_title('Performance Comparison by Prompt Template', fontsize=16)
ax.set_xlabel('Prompt Template', fontsize=14)
ax.set_ylabel('Score', fontsize=14)
ax.set_ylim(0, 1.0)
ax.set_xticks(x)
ax.set_xticklabels([name.title() for name in template_names], fontsize=12)
ax.legend(fontsize=12)

# Add value labels on bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "template_comparison.png"), dpi=300)
plt.close()


Using device: cuda for inference
Loading model from C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\reward_model_best_20250504-131132\best_model


C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating summaries with multiple strategies...

Generating with template: basic


100%|██████████████████████████████████████████████████████████████████████████| 8/8 [01:03<00:00,  7.92s/it]



Generating with template: medical


...

Evaluating fluency of 30 samples !!!


100%|████████████████████████████████████████████████████████████████████████| 54/54 [00:08<00:00,  6.19it/s]


Evaluating relevance of 30 samples !!!


100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.45it/s]



Evaluation scores are shown below:
+-------------+----------+
|  Dimensions |  Score   |
+-------------+----------+
|  coherence  | 0.690897 |
| consistency | 0.696593 |
|   fluency   | 0.627376 |
|  relevance  | 0.621353 |
|   overall   | 0.659055 |
+-------------+----------+

=== Basic Template Results ===
Average Coherence: 0.7295
Average Consistency: 0.6937
Average Fluency: 0.6645
Average Relevance: 0.6379
Overall Average: 0.6814

=== Medical Template Results ===
Average Coherence: 0.7146
Average Consistency: 0.6961
Average Fluency: 0.6429
Average Relevance: 0.5901
Overall Average: 0.6609

=== Extract Template Results ===
Average Coherence: 0.7713
Average Consistency: 0.7258
Average Fluency: 0.6461
Average Relevance: 0.6591
Overall Average: 0.7006

=== Summary Template Results ===
Average Coherence: 0.6909
Average Consistency: 0.6966
Average Fluency: 0.6274
Average Relevance: 0.6214
Overall Average: 0.6591

🏆 Best Template: Extract with Overall Score: 0.7006


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model
model_path = r"D:\kshitij-weights-folder\qwen-aloe-rl-12-4-ppo-tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Create a synthetic medical conversation
synthetic_conversation = """
[doctor] Good morning, Mrs. Johnson. How are you feeling today?
[patient] Good morning, Doctor. Not too well. I've been having this persistent cough for about 3 weeks now. It's worse at night and I'm bringing up yellowish phlegm.
[doctor] I'm sorry to hear that. Have you had any fever or chills?
[patient] Yes, I've had a low-grade fever of about 99.5°F for the past week. I've also been feeling unusually tired.
[doctor] And how about shortness of breath or chest pain?
[patient] I do get short of breath sometimes, especially when walking up stairs. No chest pain though.
[doctor] Are you taking any medications currently?
[patient] Just my regular blood pressure medicine - Lisinopril 10mg once daily. I've been taking some over-the-counter cough syrup but it's not helping much.
[doctor] Have you had any recent illnesses or been around anyone who's been sick?
[patient] My grandson had a bad cold about a month ago. I was taking care of him for a few days.
[doctor] Let me examine you. Your temperature is 99.8°F now. Blood pressure is 138/85, which is slightly elevated. Your oxygen saturation is 94%, which is a bit lower than I'd like to see.
[doctor] When I listen to your lungs, I can hear some crackles in the lower right lobe. I think we should get a chest X-ray to rule out pneumonia.
[patient] That sounds concerning. Do you think it's serious?
[doctor] It could be a case of bacterial pneumonia. I'll prescribe an antibiotic - azithromycin - for 5 days. If it is pneumonia, you should start feeling better within 48-72 hours on the antibiotics.
[doctor] I'd also like you to use this inhaler - it's albuterol - to help with the breathing when you feel short of breath. Two puffs every 4-6 hours as needed.
[patient] Should I continue with the over-the-counter cough medicine?
[doctor] You can continue with it at night if the cough is keeping you up, but it's actually good to cough during the day to clear the infection from your lungs.
[doctor] Make sure to drink plenty of fluids, get rest, and come back if you don't start feeling better in a few days, or if you develop high fever, severe shortness of breath, or chest pain.
[patient] Thank you, Doctor. I'll follow your instructions.
[doctor] I'll see you for a follow-up in one week, but call if anything worsens before then.
"""

# Use the "medical" template which performed best in your evaluation
prompt_template = "Generate a concise medical summary of the conversation:\n\n"
prompt = f"{prompt_template}{synthetic_conversation}"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate summary with beam search for better quality
with torch.no_grad():
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        do_sample=False,
        num_beams=5,
        max_new_tokens=128,
        no_repeat_ngram_size=3
    )

# Decode the output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Extract just the generated part
if prompt in generated_text:
    summary = generated_text[generated_text.find(prompt) + len(prompt):]
else:
    summary = generated_text

# Display results
print("\n=== Input Medical Conversation ===")
print(synthetic_conversation[:300] + "...")
print("\n=== Generated Medical Summary ===")
print(summary)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards:   0%|                                                       | 0/4 [00:00<?, ?it/s]


OSError: The paging file is too small for this operation to complete. (os error 1455)